Here we want to simply check the german data first and give some function to make it suitable for the CNN framework

In [11]:
import pandas as pd
from torch.utils.data import Dataset
import torch
import os
import numpy as np
import matplotlib.pyplot as plt

In [12]:
german_data = pd.read_csv('data/german data/german.data',sep= " ")
column_names = [
    "Status_of_existing_checking_account", "Duration_in_month", "Credit_history",
    "Purpose", "Credit_amount", "Savings_account/bonds", "Present_employment_since",
    "Installment_rate_in_percentage_of_disposable_income", "Personal_status_and_sex",
    "Other_debtors/guarantors", "Present_residence_since", "Property",
    "Age_in_years", "Other_installment_plans", "Housing", "Number_of_existing_credits_at_this_bank",
    "Job", "Number_of_people_being_liable_to_provide_maintenance_for",
    "Telephone", "Foreign_worker","Good/Bad"
]
german_data.columns = column_names

这里我们不能继续使用颜色对该数据进行标签化，因为我们的数据是表格而不是图片形式。
对应benefit的标签：Good
对应绿色/红色的标签：sex（此处只考虑性别）
首先检验是否存在不公平的现象：对于personal status and sex， 我们已知A92和A95代表女性

In [3]:
# 筛选 Personal_status_and_sex 为 "A92" 或 "A95"（女性） 且 Good/Bad 为 1 的行
filtered_data = german_data[
    ((german_data["Personal_status_and_sex"] == "A92") | 
    (german_data["Personal_status_and_sex"] == "A95")) & 
    (german_data["Good/Bad"] == 1)
]
filtered_count_female = filtered_data.shape[0]  # 获取行数
filtered_count_female

201

In [4]:
# 筛选 Personal_status_and_sex 为 男性 且 Good/Bad 为 1 的行
total_good = len(german_data[german_data['Good/Bad']== 1])
filtered_count_male = total_good - filtered_count_female
filtered_count_male

498

In [15]:
total_bad = len(german_data[german_data['Good/Bad']== 2])
print(total_bad)
filtered_data_bad = german_data[
    ((german_data["Personal_status_and_sex"] == "A92") | 
    (german_data["Personal_status_and_sex"] == "A95")) & 
    (german_data["Good/Bad"] == 2)
]
filtered_count_female_bad = filtered_data.shape[0]  # 获取行数
filtered_count_female_bad / total_bad

300


0.67

In [8]:
female_prob = filtered_count_female/total_good
male_prob = filtered_count_male / total_good
print(female_prob)
print(male_prob)
female_prob + male_prob


0.2875536480686695
0.7124463519313304


1.0

In [15]:
female_data = german_data[
    ((german_data["Personal_status_and_sex"] == "A92") | 
    (german_data["Personal_status_and_sex"] == "A95"))]
total_female = female_data.shape[0]
# total_female
prob_fem = filtered_count_female / total_female
prob_fem

0.6483870967741936

In [16]:
total = len(german_data)
total_male = total - total_female
prob_mal = filtered_count_male / total_male
prob_mal

0.7227866473149492

we can see that there is a huge difference between female and male on the number of goods
thus we take good/bad as 1 be the benefit groups, and =2 for the not benefit groups
sex = male as green in the previous paper, sex = female as red in the previous paper

In [7]:
german_data.columns

Index(['Status_of_existing_checking_account', 'Duration_in_month',
       'Credit_history', 'Purpose', 'Credit_amount', 'Savings_account/bonds',
       'Present_employment_since',
       'Installment_rate_in_percentage_of_disposable_income',
       'Personal_status_and_sex', 'Other_debtors/guarantors',
       'Present_residence_since', 'Property', 'Age_in_years',
       'Other_installment_plans', 'Housing',
       'Number_of_existing_credits_at_this_bank', 'Job',
       'Number_of_people_being_liable_to_provide_maintenance_for', 'Telephone',
       'Foreign_worker', 'Good/Bad'],
      dtype='object')

In [17]:
# def color_grayscale_arr(arr, red=True):
#     """Converts grayscale image to either red or green"""
#     assert arr.ndim == 2
#     dtype = arr.dtype
#     h, w = arr.shape
#     arr = np.reshape(arr, [h, w, 1])
#     if red:
#         arr = np.concatenate([arr,
#                               np.zeros((h, w, 2), dtype=dtype)], axis=2)
#     else:
#         arr = np.concatenate([np.zeros((h, w, 1), dtype=dtype),
#                               arr,
#                               np.zeros((h, w, 1), dtype=dtype)], axis=2)
#     return arr

# similar to the definition before we have:
def map_gender(value):
        if value in ["A91", "A93", "A94"]:
            return "male"
        elif value in ["A92","A95"]:
            return "female"
        else:
            return "unknown"
def map_good(value):
     if value == 2:
          return 0
     else:
          return 1
def sex_change(data):
    """ converts personal status data to either male or female, and change good to 1 , bad to 0"""
    data["Personal_status_and_sex"] = data["Personal_status_and_sex"].apply(map_gender)
    data["Good/Bad"] = data["Good/Bad"].apply(map_good)
    return data


In [18]:
german_data = sex_change(german_data)
german_data["sex"] = german_data["Personal_status_and_sex"]

In [19]:
german_data.drop("Personal_status_and_sex",axis=1)

,Status_of_existing_checking_account,Duration_in_month,Credit_history,Purpose,Credit_amount,Savings_account/bonds,Present_employment_since,Installment_rate_in_percentage_of_disposable_income,Other_debtors/guarantors,Present_residence_since,...,Age_in_years,Other_installment_plans,Housing,Number_of_existing_credits_at_this_bank,Job,Number_of_people_being_liable_to_provide_maintenance_for,Telephone,Foreign_worker,Good/Bad,sex
0,A12,48,A32,A43,5951,A61,A73,2,A101,2,...,22,A143,A152,1,A173,1,A191,A201,0,female
1,A14,12,A34,A46,2096,A61,A74,2,A101,3,...,49,A143,A152,1,A172,2,A191,A201,1,male
2,A11,42,A32,A42,7882,A61,A74,2,A103,4,...,45,A143,A153,1,A173,2,A191,A201,1,male
3,A11,24,A33,A40,4870,A61,A73,3,A101,4,...,53,A143,A153,2,A173,2,A191,A201,0,male
4,A14,36,A32,A46,9055,A65,A73,2,A101,4,...,35,A143,A153,1,A172,2,A192,A201,1,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,A14,12,A32,A42,1736,A61,A74,3,A101,4,...,31,A143,A152,1,A172,1,A191,A201,1,female
995,A11,30,A32,A41,3857,A61,A73,4,A101,4,...,40,A143,A152,1,A174,1,A192,A201,1,male
996,A14,12,A32,A43,804,A61,A75,4,A101,4,...,38,A143,A152,1,A173,1,A191,A201,1,male
997,A11,45,A32,A43,1845,A61,A73,4,A101,4,...,23,A143,A153,1,A173,1,A192,A201,0,male


In [20]:
def map_sex(value):
     if value == "female":
          return 0
     else:
          return 1
german_data["sex"] = german_data["sex"].apply(map_sex)

In [21]:
german_data

,Status_of_existing_checking_account,Duration_in_month,Credit_history,Purpose,Credit_amount,Savings_account/bonds,Present_employment_since,Installment_rate_in_percentage_of_disposable_income,Personal_status_and_sex,Other_debtors/guarantors,...,Age_in_years,Other_installment_plans,Housing,Number_of_existing_credits_at_this_bank,Job,Number_of_people_being_liable_to_provide_maintenance_for,Telephone,Foreign_worker,Good/Bad,sex
0,A12,48,A32,A43,5951,A61,A73,2,female,A101,...,22,A143,A152,1,A173,1,A191,A201,0,0
1,A14,12,A34,A46,2096,A61,A74,2,male,A101,...,49,A143,A152,1,A172,2,A191,A201,1,1
2,A11,42,A32,A42,7882,A61,A74,2,male,A103,...,45,A143,A153,1,A173,2,A191,A201,1,1
3,A11,24,A33,A40,4870,A61,A73,3,male,A101,...,53,A143,A153,2,A173,2,A191,A201,0,1
4,A14,36,A32,A46,9055,A65,A73,2,male,A101,...,35,A143,A153,1,A172,2,A192,A201,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,A14,12,A32,A42,1736,A61,A74,3,female,A101,...,31,A143,A152,1,A172,1,A191,A201,1,0
995,A11,30,A32,A41,3857,A61,A73,4,male,A101,...,40,A143,A152,1,A174,1,A192,A201,1,1
996,A14,12,A32,A43,804,A61,A75,4,male,A101,...,38,A143,A152,1,A173,1,A191,A201,1,1
997,A11,45,A32,A43,1845,A61,A73,4,male,A101,...,23,A143,A153,1,A173,1,A192,A201,0,1


In [23]:
file_path = "/Users/liuxinhao/Desktop/fair german data/data/German_data.csv"
german_data.to_csv(file_path,index= False)

接下来创建一个类，适用于接下来的分析

In [75]:
class GermanDataset(Dataset):
    def __init__(self, root = './data/German_data', train=True, valid=False, transform=None, target_transform=None):
        # here do not define the pos_class_thresh since we already have good/bad
        # train: train set
        super(GermanDataset, self).__init__(root, transform=transform, target_transform=target_transform)
        self.root = root
        self.prepare_german(is_train=train, is_valid=valid) #根据给定的参数生成训练集、验证集或测试集
        assert not (train and valid)
        if train:
            self.data_label_tuples = torch.load(
                os.path.join(self.root, 'train.pt'))
        elif not train and not valid:  # test
            self.data_label_tuples = torch.load(
                os.path.join(self.root, 'test.pt'))
        else:  # valid set
            self.data_label_tuples = torch.load(
                os.path.join(self.root, 'valid.pt'))
    def __getitem__(self, index):
        item = self.data[index]
        target = item["Good/Bad"] #target
        sensitive = item['Personal_status_and_sex'] # gender

        if self.transform:
            item = self.transform(item)
        if self.target_transform:
            target = self.target_transform(target)

        return item, sensitive, target

    def __len__(self):
        return len(self.data)
    
    def prepare_german(self, is_train=True, is_valid=False):
        if is_train or is_valid:
            # Load German dataset
            german_data = pd.read_csv(os.path.join(self.root, 'German_data.csv'))

            # Here you can add your logic to prepare the dataset
            # For example, adding noise, inducing relationship between gender and label, etc.
            # 将German数据集转换为类似于Colored MNIST的形式，即为每个样本添加敏感性信息和目标标签

            train_set = []
            valid_set = []
            for idx, row in german_data.iterrows():
                # Assign a binary label based on some condition (e.g., credit rating)
                binary_label = 1 if row['Good/Bad'] == 1 else 0

                # Induce relationship between gender and label
                # 假设根据性别不同，好坏信用的分布有所差异
                if row['Personal_status_and_sex'] == 'male':
                    if np.random.uniform() < 0.7227866473149492:  # 男性中，好信用的概率为0.7227866473149492
                        good = 1  # 好信用
                    else:
                        good = 0  # 坏信用
                else:  # 女性
                    if np.random.uniform() < 0.6483870967741936:  # 女性中，好信用的概率为0.6483870967741936
                        good = 1  # 好信用
                    else:
                        good = 0  # 坏信用

                # Add the sample to the appropriate set (train or valid)
                if idx > 500:  # 前500个样本作为验证集，其余作为训练集
                    valid_set.append((row, good, binary_label))
                else:
                    train_set.append((row, good, binary_label))

            # Save the datasets as .pt files
            if not os.path.exists(self.root):
                os.makedirs(self.root)
            if is_train:
                torch.save(train_set, os.path.join(self.root, 'train.pt'))
                torch.save(valid_set, os.path.join(self.root, 'valid.pt'))
            else:  # test
                # Here you can prepare the test set if needed
                torch.save(train_set, os.path.join(self.root, 'test.pt'))
         
    
        

    
    
    

In [ ]:
def plot_dataset_samples(dataset, save_as):
    # 从数据集中获取一些样本
    num_samples = min(50, len(dataset))
    samples = dataset[:num_samples]

    # 确保保存路径存在
    os.makedirs(os.path.dirname(save_as), exist_ok=True)

    # 打印一些样本的数据
    for i, (item, sensitive, target) in enumerate(samples):
        print(f"Sample {i+1}:")
        print("Features:")
        print(item)  # 输出样本的特征
        print("Sensitive information (Gender):", sensitive)
        print("Target (Good/Bad):", target)
        print("-----------------------------------")

    # 绘制一些样本的数据分布（例如，Good/Bad 的分布）
    targets = [target for _, _, target in samples]
    labels, counts = np.unique(targets, return_counts=True)
    plt.bar(labels, counts, align='center')
    plt.xlabel('Target')
    plt.ylabel('Count')
    plt.title('Distribution of Targets (Good/Bad)')
    plt.xticks(labels)
    plt.grid(axis='y')
    plt.tight_layout()

    # 保存图像
    plt.savefig(save_as, dpi=400)

    # 清空图形并关闭图
    plt.cla()
    plt.clf()
    plt.close()

In [16]:
data = pd.read_csv("data/German_data.csv")

In [17]:
# 假设 'Good/Bad' 是第1列，'Sensitive' 是第2列，其余的是特征
good_bad = data['Good/Bad']
sensitive = data['sex']
features = data.drop(columns=['Good/Bad', 'sex'])

# 确认每部分数据的形状
print(f"Good/Bad shape: {good_bad.shape}")
print(f"Sensitive shape: {sensitive.shape}")
print(f"Features shape: {features.shape}")


Good/Bad shape: (999,)
Sensitive shape: (999,)
Features shape: (999, 56)
